# Paper

[Hierarchical Adaptive Voxel-guided Sampling for Real-time Applications in Large-scale Point Clouds](https://arxiv.org/abs/2305.14306)

# Code

In [1]:
import numpy as np
import laspy
import matplotlib.pyplot as plt
from multiprocessing import Pool

In [38]:
P = np.random.rand(2_000_000,3)#laspy.read("../datasets/unknownPL_stockpile.laz").xyz
P.shape

(2000000, 3)

In [39]:
(P.min(), P.max())

(1.6101954480873815e-07, 0.9999999593075063)

In [40]:
v = np.repeat((P.max() + P.min())/2**6, 3)
v

array([0.015625, 0.015625, 0.015625])

# Sparse Voxelization via Voxel-Hashing

## Voxels Coordinates Computation

In [41]:
def compute_voxel_coordinates(P: np.ndarray, v: np.ndarray) -> np.ndarray:
    return np.floor(P / v).astype(int)

voxel_coordinates = compute_voxel_coordinates(P, v)
voxel_coordinates.shape

(2000000, 3)

In [42]:
voxel_coordinates[0:10]

array([[45, 35,  9],
       [47,  9, 13],
       [17, 34, 23],
       [54, 33, 31],
       [ 0, 58, 33],
       [10, 19,  1],
       [31, 38,  2],
       [30,  4,  8],
       [30, 52,  4],
       [10, 15,  0]])

## Voxel-Hashing + Closest Center Strategy

In [43]:
def center_closest_point_sampling(P: np.ndarray, v: np.ndarray) -> np.array:
    
    points_voxel_coordinates = compute_voxel_coordinates(P, v)
    
    T = {}

    for point_index, point_voxel_coordinate in enumerate(points_voxel_coordinates):
        
        point = P[point_index]
        
        distance_to_voxel_center = np.linalg.norm(point - np.multiply(v, (point_voxel_coordinate + 0.5)), ord=1)
        
        hashable_voxel_coordinates = tuple(point_voxel_coordinate)
        
        if T.get(hashable_voxel_coordinates) is not None:
            existing_distance = T[hashable_voxel_coordinates][1]
            if distance_to_voxel_center < existing_distance:
                T[hashable_voxel_coordinates] = (point_index, distance_to_voxel_center)
        else:
            T[hashable_voxel_coordinates] = (point_index, distance_to_voxel_center)

    return np.array([ i for (i, _) in T.values() ])

sample_indexes = center_closest_point_sampling(P, v)
print(f"sample_indexes.shape: {sample_indexes.shape}")
print(f"sample_indexes (First 5 values): {list(sample_indexes[:5])}")

sample_indexes.shape: (262004,)
sample_indexes (First 5 values): [922437, 1973512, 1940974, 432141, 4]


# Adaptive Voxel Search

In [44]:
def count_voxels(points: np.ndarray, v: np.ndarray) -> int:
    
    voxel_coordinates = compute_voxel_coordinates(points, v)
    
    T = {}
    
    for voxel_coordinate in voxel_coordinates:
        T[tuple(voxel_coordinate)] = 1
    
    return len(T.keys())

def adaptive_voxel_search(points: np.ndarray,
                          sampling_number: int, 
                          tolerance: float = 0.1,
                          max_iter: int = 20) -> np.ndarray:
    print(f"adaptive_voxel_search with sampling_number: {sampling_number}, tolerance: {tolerance}")
    
    l = points.min()
    r = points.max()
    
    voxel_size = (l + r) / 2
    
    for iter in range(max_iter):
        m_s = count_voxels(points, np.repeat(voxel_size, 3))
        print(f"voxel_size: {voxel_size}, m_s: {m_s}")
        
        if m_s < sampling_number:
            r = voxel_size
        elif m_s > (1+tolerance)*sampling_number:
            l = voxel_size
        else:
            return np.repeat(voxel_size, 3)

        voxel_size = (l + r) / 2
    
    return np.repeat(voxel_size, 3)

In [45]:
v_temp = adaptive_voxel_search(P, 4096)
v_temp

adaptive_voxel_search with sampling_number: 4096, tolerance: 0.1
voxel_size: 0.5000000601635255, m_s: 8
voxel_size: 0.25000011059153515, m_s: 64
voxel_size: 0.12500013580553998, m_s: 512
voxel_size: 0.0625001484125424, m_s: 4096


array([0.06250015, 0.06250015, 0.06250015])

# HAV Sampler

In [46]:
def compute_hav_sampling_iteration(P_l: np.ndarray, m_l: int, avs_tolerance: float, avs_max_iter: int) -> list:
    print(f"m_l: {m_l}")
        
    v = adaptive_voxel_search(P_l, m_l, tolerance=avs_tolerance, max_iter=avs_max_iter)
    P_l_prime = center_closest_point_sampling(P_l, v)
    
    layer_points = P_l[P_l_prime]
    print(f"layer_points.shape: {layer_points.shape}")
    
    return list(layer_points)

In [54]:
def hav_sampler(points: np.ndarray, 
                sampling_layers: list = [4**6, 4**7, 4**8, 4**9],
                avs_tolerance: float = 0.1,
                avs_max_iter: int = 20) -> np.ndarray:
    global P_l
    P_l = points
    
    args = [ (P_l, m_l, avs_tolerance, avs_max_iter) for m_l in sampling_layers ]
    
    layers_points = []
    
    with Pool(processes=None) as pool:
        layers_points = pool.starmap(compute_hav_sampling_iteration, args)

    layers_points = [ item for sublist in layers_points for item in sublist ]
    
    return np.array(layers_points)[:sum(sampling_layers)]

In [55]:
sample = hav_sampler(P)

m_l: 4096
adaptive_voxel_search with sampling_number: 4096, tolerance: 0.1
m_l: 16384
adaptive_voxel_search with sampling_number: 16384, tolerance: 0.1
m_l: 65536
adaptive_voxel_search with sampling_number: 65536, tolerance: 0.1
m_l: 262144
adaptive_voxel_search with sampling_number: 262144, tolerance: 0.1
voxel_size: 0.5000000601635255, m_s: 8
voxel_size: 0.5000000601635255, m_s: 8
voxel_size: 0.5000000601635255, m_s: 8
voxel_size: 0.5000000601635255, m_s: 8
voxel_size: 0.25000011059153515, m_s: 64
voxel_size: 0.25000011059153515, m_s: 64
voxel_size: 0.25000011059153515, m_s: 64
voxel_size: 0.25000011059153515, m_s: 64
voxel_size: 0.12500013580553998, m_s: 512
voxel_size: 0.12500013580553998, m_s: 512
voxel_size: 0.12500013580553998, m_s: 512
voxel_size: 0.12500013580553998, m_s: 512
voxel_size: 0.0625001484125424, m_s: 4096
voxel_size: 0.0625001484125424, m_s: 4096
voxel_size: 0.0625001484125424, m_s: 4096
voxel_size: 0.0625001484125424, m_s: 4096
voxel_size: 0.0312501547160436, m_s:

In [56]:
sample.shape

(348160, 3)

In [57]:
sum(sampling_layers)

348160

## Save sample to LAS

In [58]:
header = laspy.LasHeader(point_format=3, version="1.4")
header.offsets = np.min(sample, axis=0)
header.scales = np.array([0.1, 0.1, 0.1])

las = laspy.LasData(header)

las.x = sample[:, 0]
las.y = sample[:, 1]
las.z = sample[:, 2]

las.write("sample.las")